In [1]:
import pyodbc
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')

# QUERY LINES: PICKING - PACKING - BATCHING

In [2]:
# Connect to the Database
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

In [3]:
df = pd.read_sql_query("DECLARE @from date, @to date;\
    SELECT @from = getdate() ,@to = dateadd(day, 1, getdate());\
\
    select t.completedDateTime Date, w.name Warehouse, p.name Project,\
        oc.name OperationCode, t.employee Employee\
    from datex_footprint.Tasks t\
        left join datex_footprint.Warehouses w\
            on w.id = t.warehouseId\
        left join datex_footprint.Projects p\
            on p.id = t.projectId\
        join datex_footprint.OperationCodes oc\
            on oc.id = t.operationCodeId\
        where t.completedDateTime >= @from AND t.completedDateTime < @to AND\
        (t.employee != 'FootPrintService') AND (t.employee != 'Foot Print Service') AND\
        t.statusId = 2 AND t.operationCodeId in (8, 39, 2067)", cnxn)

In [4]:
# Adding Hours
df['Hour'] = pd.to_datetime(df['Date']).dt.hour
df.head()

,Date,Warehouse,Project,OperationCode,Employee,Hour
0,2022-05-13 07:47:50.063,20,Reliable 1 Laboratories LLC,Picking,Marzetta Bracken,7
1,2022-05-13 07:53:14.877,20,Reliable 1 Laboratories LLC,Picking,Marzetta Bracken,7
2,2022-05-13 07:34:18.843,22,"NBTY Global, Inc.",Picking,William Trevino Trevino,7
3,2022-05-13 07:51:05.860,22,"NBTY Global, Inc.",Picking,William Trevino Trevino,7
4,2022-05-13 07:33:14.700,22,"NBTY Global, Inc.",Picking,William Trevino Trevino,7


In [5]:
# Activity Function By Warehouse
def activity (warehouse, operationCode, df):
    # Filter by warehouse
    df2 = df[(df['Warehouse'] == warehouse) & (df['OperationCode'] == operationCode)]
    
    # Grouped by Employee and Hour
    df2 = df2.groupby(['Employee', 'Hour']).count()[['OperationCode']]
    df2.reset_index(inplace=True)
    
    # Pivot
    df2 = df2.pivot(index='Employee', columns='Hour', values='OperationCode')
    df2 = df2.fillna(0)
    df2 = df2.rename_axis(None, axis=1)
    
    # Save into JSON
    df2 = df2.T
    file_name = f'data/W{warehouse}_{operationCode}.json'
    df2.to_json(file_name)

In [6]:
activity('10', 'Picking', df)
activity('10', 'Rate And Ship', df)
activity('10', 'Batch Move', df)

In [7]:
# # Close connection
cnxn.close()

# QUERY ORDERS: PICKING - PACKING - BATCHING

In [8]:
# Connect to the Database
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

In [9]:
# TODAY ORDERS
df_today = pd.read_sql_query("DECLARE @from date, @to date;\
    SELECT @from = getdate() ,@to = dateadd(day, 1, getdate());\
\
    select distinct w.name Warehouse, oc.name OperationCode,\
        COUNt(DISTINCT (ISNULL(s.lookupCode, t.shipmentId))) Shipment\
    from datex_footprint.Tasks t\
        left join datex_footprint.Warehouses w\
            on w.id = t.warehouseId\
        join datex_footprint.OperationCodes oc\
            on oc.id = t.operationCodeId\
        left join datex_footprint.PickSlips ps\
            on ps.id = t.pickSlipId\
        left join datex_footprint.Waves wa\
            on wa.id = ps.waveId\
        left join datex_footprint.shipments s\
            on s.waveId = wa.id\
        where t.completedDateTime >= @from AND t.completedDateTime < @to AND\
        (t.employee != 'FootPrintService') AND (t.employee != 'Foot Print Service') AND\
        t.statusId = 2 AND t.operationCodeId in (8, 39, 2067)\
        group by w.name, oc.name", cnxn)

In [10]:
# YESTEDAY ORDERS
df_yesterday = pd.read_sql_query("DECLARE @from date, @to date;\
    SELECT @from = dateadd(day, -1, getdate()) ,@to = getdate();\
\
    select distinct w.name Warehouse, oc.name OperationCode,\
        COUNt(DISTINCT (ISNULL(s.lookupCode, t.shipmentId))) Shipment\
    from datex_footprint.Tasks t\
        left join datex_footprint.Warehouses w\
            on w.id = t.warehouseId\
        join datex_footprint.OperationCodes oc\
            on oc.id = t.operationCodeId\
        left join datex_footprint.PickSlips ps\
            on ps.id = t.pickSlipId\
        left join datex_footprint.Waves wa\
            on wa.id = ps.waveId\
        left join datex_footprint.shipments s\
            on s.waveId = wa.id\
        where t.completedDateTime >= @from AND t.completedDateTime < @to AND\
        (t.employee != 'FootPrintService') AND (t.employee != 'Foot Print Service') AND\
        t.statusId = 2 AND t.operationCodeId in (8, 39, 2067)\
        group by w.name, oc.name", cnxn)

In [11]:
# Rename Columns
df_today.rename(columns={'Shipment': 'Today'}, inplace=True)
df_yesterday.rename(columns={'Shipment': 'Yesterday'}, inplace=True)

In [12]:
# Merge dataframes
new_df = pd.merge(df_today, df_yesterday,  how='outer', left_on=['Warehouse','OperationCode'], right_on = ['Warehouse','OperationCode'])

In [13]:
def orders (warehouse, df):
    df2 = df[df['Warehouse'] == warehouse]
    df2 = df2.pivot(index='Warehouse', columns='OperationCode', values=['Today', 'Yesterday']).fillna(0)
    df2 = df2.T
    
    display(df2)
    
    file_name = f'data/W{warehouse}_Orders.json'
    df2.to_json(file_name)

In [14]:
orders('10', new_df)

Warehouse                   10
          OperationCode       
Today     Batch Move       0.0
          Picking          0.0
          Rate And Ship    0.0
Yesterday Batch Move     227.0
          Picking        297.0
          Rate And Ship  110.0

In [15]:
# # Close connection
cnxn.close()

In [16]:
# def orders (warehouse, df):
#     df2 = df[df['Warehouse'] == warehouse]
    
#     df2 = df2.pivot(index='Warehouse', columns='OperationCode', values='Today').fillna(0)
#     df2 = df2.rename_axis(None, axis=1)
    
#     df2 = df2.T
    
#     display(df2)
    
#     file_name = f'data/W{warehouse}_Orders.json'
#     df2.to_json(file_name)

# QUERY 02: Open Outbounds WAREHOUSE 10

In [17]:
# Connect to the Database

# cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
#                         "Server=wd02;"
#                         "Database=FootPrint;"
#                         "Trusted_Connection=yes;")

In [18]:
# df_02 = pd.read_sql_query("SELECT DISTINCT w.name warehouse, p.name project, s.lookupCode shipment_number, s.createdSysDateTime\
#     FROM datex_footprint.shipments s\
#     LEFT JOIN datex_footprint.ShipmentOrderLookup sol\
#         on sol.shipmentId = s.id\
#     LEFT JOIN datex_footprint.orders o\
#         on o.id = sol.orderId\
#     LEFT JOIN datex_footprint.Projects p\
#         on p.id = o.projectId\
#     LEFT JOIN datex_footprint.warehouses w\
#         on w.id = isnull(s.actualWarehouseId, s.expectedWarehouseId)\
#     WHERE s.statusId in (1, 2, 4) and s.typeId = 2\
#     ORDER BY createdSysDateTime", cnxn)

In [19]:
# df_02.head()

In [20]:
# df_02['age_order'] = pd.Timestamp.now() - df_02['createdSysDateTime']
# df_02.head()

In [21]:
# df_02['age_order_days'] = df_02['age_order'].dt.components['days']
# df_02['age_order_hours'] = df_02['age_order'].dt.components['hours']
# df_02.head()

In [22]:
# Open Orders less than 24 hours

# df_02 = df_02.loc[(df_02['age_order_days'] == 0) & (df_02['warehouse'] == '10')]
# df_02.head()

In [23]:
# Bins per hour

# bins =[-1,4,8,12,16,24]
# bin_names = ['4 Hours', '8 Hours', '12 Hours', '16 Hours', '24 Hours']

# df_02['age_group'] = pd.cut(df_02['age_order_hours'], bins, labels=bin_names)

In [24]:
# df_02_grouped = df_02.groupby(['age_group'])[['project']].count()

In [25]:
# df_02_grouped = df_02_grouped.rename(columns={'project': 'orders'})

### Save in JSON File

In [26]:
# open_outbounds = df_02_grouped
# open_outbounds.to_json('data/open_outbounds.json')
# open_outbounds

In [27]:
# df_02_grouped = df_02_grouped.iloc[::-1]
# df_02_grouped.plot.barh()

In [28]:
# Close connection

# cnxn.close()

# QUERY 3: Picking Performance (Orders and Waves)

In [29]:
# Connect to the Database

# cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
#                         "Server=wd02;"
#                         "Database=FootPrint;"
#                         "Trusted_Connection=yes;")

In [30]:
# df03 = pd.read_sql_query("DECLARE @from date, @to date;\
#                           SELECT @from = getdate(), @to = dateadd(day, 1, getdate());\
#     SELECT t.completedDateTime Date, oc.name Operation, w.name Warehouse,\
#            p.name Project, m.lookupCode Material, employee Employee, s.lookupCode Shipment, isnull(s.waveId, ps.waveId) Wave,\
#            ordersPerWave.shipmentsCount, lc.name Location\
#     FROM datex_footprint.Tasks t\
#         join datex_footprint.OperationCodes oc\
#             on oc.id = t.operationCodeId\
#         left join datex_footprint.Warehouses w\
#             on w.id = t.warehouseId\
#         left join datex_footprint.Projects p\
#             on p.id = t.projectId\
#         left join datex_footprint.Materials m\
#             on m.id = t.materialId\
#         left join datex_footprint.Shipments s\
#             on s.id = t.shipmentId\
#         left join datex_footprint.PickSlips ps\
#             on ps.id = t.pickSlipId\
#         left join datex_footprint.LocationContainers lc\
#             on lc.id = t.actualSourceLocationId\
#         outer apply (\
#             select\
#                 count(distinct tt.shipmentId) shipmentsCount\
#                 from datex_footprint.waves ww\
#                     inner join datex_footprint.PickSlips ppss\
#                     inner join datex_footprint.tasks tt\
#                         on tt.operationCodeId = 8 and tt.pickSlipId = ppss.id\
#                         on ppss.waveId = ww.id\
#                 where ww.id = isnull(s.waveId, ps.waveId)) ordersPerWave\
#     where t.completedDateTime >= @from and t.completedDateTime < @to and t.statusId = 2\
#     and t.operationCodeId in (8, 39, 2064, 2065, 2066) and t.employee != 'Foot Print Service'\
#     order by s.lookupCode, t.completedDateTime", cnxn)

In [31]:
# df03.tail(3)

In [32]:
# Order Columns

# df03 = df03[['Warehouse', 'Project', 'Wave', 'Shipment', 'Date', 'Operation', 'Employee',
#              'Location', 'shipmentsCount', 'Material']]

### Filter by Shipments

In [33]:
# Filter by Orders only

# df03_orders = df03.loc[(df03['Operation']=='Picking') |
#                        (df03['Operation']=='Scanned Shipment') |
#                        (df03['Operation']=='Picking Completed')]

# df03_orders = df03_orders[['Warehouse', 'Project', 'Shipment', 'Date', 'Operation', 'Employee', 'Location']]

# df03_orders.head()

In [34]:
# Create a new column with the time difference between rows

# df03_orders['PickingTime'] = df03_orders['Date'].diff()
# df03_orders.head()

In [35]:
# Add Hour, Minute and Second columns

# df03_orders['hour'] = df03_orders['PickingTime'].dt.components['hours']
# df03_orders['minute'] = df03_orders['PickingTime'].dt.components['minutes']
# df03_orders['second'] = df03_orders['PickingTime'].dt.components['seconds']
# df03_orders.head()

In [36]:
# Set all the Scanned Shipment hour, minute, time to ZERO

# df03_orders_modified = df03_orders

# df03_orders_modified.loc[df03_orders_modified.Operation == 'Scanned Shipment', 'hour'] = 0.0
# df03_orders_modified.loc[df03_orders_modified.Operation == 'Scanned Shipment', 'minute'] = 0.0
# df03_orders_modified.loc[df03_orders_modified.Operation == 'Scanned Shipment', 'second'] = 0.0

# df03_orders_modified.loc[df03_orders_modified['Operation'] == 'Scanned Shipment'].head()

In [37]:
# df03_orders_modified['Time'] = df03_orders_modified['hour']*60 + df03_orders_modified['minute'] + df03_orders_modified['second']/60
# df03_orders_modified.head()

In [38]:
# Save in JSON Format for HTML

# df03_group = df03_orders_modified.groupby(['Warehouse', 'Project', 'Shipment', 'Operation', 'Date'])[['Time']].sum().sort_values(['Warehouse', 'Project', 'Shipment', 'Date'])
# df03_group.to_json('data/performance.json', orient='table')
# df03_group.head(20)

In [39]:
# Remove Operation 'Scanned Shipment'

# df03_excluding_Scanned = df03_orders_modified.loc[df03_orders_modified['Operation'] != 'Scanned Shipment']
# df03_excluding_Scanned.head()

In [40]:
# Save in CSV Format for Tableau

# df03_group2 = df03_excluding_Scanned.groupby(['Warehouse', 'Project', 'Employee', 'Shipment', 'Operation', 'Date'])[['Time']].sum().sort_values(['Warehouse', 'Project', 'Employee', 'Shipment', 'Date'])
# df03_group2.to_csv('data/performance.csv')
# df03_group2.head(20)

In [41]:
# Close connection

# cnxn.close()